In [1]:
import pandas as pd, numpy as np
from datetime import timedelta, datetime as dt
import warnings
from IPython.display import clear_output, display

#Ignore alerts
warnings.filterwarnings("ignore")

In [2]:
# Read recipe inputs - Telemetria
date_col_alertas = 'data_inicio'
gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy = pd.read_csv("GR_Alertas_Telemetria_prepared2_sem_manutencao_dummy.csv", sep=";")
gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df = pd.DataFrame(gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy)
gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df['MATRÍCULA'] = gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df['MATRÍCULA'].astype(str)
gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df[date_col_alertas] = gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df[date_col_alertas].astype(int)
gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df.fillna('', inplace=True)

# gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df.head(2)

In [3]:
# Read recipe inputs - Acidentes
date_col_acidentes = 'data_sinistro'
gr_Sinistros_Acidentes_arrumado = pd.read_csv("GR_Sinistros_Acidentes_arrumado.csv", sep=";")
gr_Sinistros_Acidentes_arrumado_df = pd.DataFrame(gr_Sinistros_Acidentes_arrumado)
gr_Sinistros_Acidentes_arrumado_df['MATRÍCULA'] = gr_Sinistros_Acidentes_arrumado_df['MATRÍCULA'].astype(str)
gr_Sinistros_Acidentes_arrumado_df[date_col_acidentes] = gr_Sinistros_Acidentes_arrumado_df[date_col_acidentes].astype(int)
gr_Sinistros_Acidentes_arrumado_df.fillna('', inplace=True)

# gr_Sinistros_Acidentes_arrumado_df.head()

In [4]:
# Functions
def get_drivers_windowed_events(df_info, end_date, driver_id_col, date_col_info, row_filter, window):
    init_date = int(dt.strftime(dt.strptime(str(end_date), '%Y%m%d') - timedelta(days=window), '%Y%m%d'))
    query_str = f"{init_date} <= {date_col_info} <= {end_date} and {driver_id_col} == '{row_filter}'"
    df = df_info.query(query_str)
    df.replace('', 0, inplace=True)
    return df


In [5]:
# Shorter variables
df_acidentes = gr_Sinistros_Acidentes_arrumado_df
df_alertas = gr_Alertas_Telemetria_prepared2_sem_manutencao_dummy_df

# Parameters
window = 360 #days

In [6]:
# Counting definitions
alertas_cols = list(df_alertas.columns)
prefixes_to_count = ['classe_', 'velocidade_']

cols_to_add_in_acidentes = [c for c in alertas_cols if any(c.startswith(p) for p in prefixes_to_count)]


In [7]:
df_count = pd.DataFrame('', index=df_acidentes.index, columns=['SIN'] + cols_to_add_in_acidentes)
df_count['SIN'] = df_acidentes['SIN']
# df_count

In [8]:
df_acidentes.iloc[1271]

SIN                                                                                      9112020
MATRÍCULA                                                                                    N/I
data_hora_do_sinistro_parsed                                            2020-05-30T19:41:00.000Z
data_sinistro                                                                           20200530
dia_da_semana                                                                                  6
CAVALO                                                                                   FDZ0316
UF                                                                                            SP
CIDADE                                                                                CAFELÂNDIA
REINCIDÊNCIA SINISTRO                                                                          0
PERÍODO                                                                                    NOITE
LOCAL OCORRÊNCIA              

In [10]:
index_list = df_acidentes.index
drivers_reg = 'MATRÍCULA'

# Counting alerts for each event
for index in index_list:

    # if index > 0:
    #     break

    if index%10 == 0:
        clear_output(wait=True)
        print(f'\r{index} of {max(index_list)}')  # use display(f) if you encounter performance issues

    end_date = df_acidentes[date_col_acidentes][index]
    driver_id = df_acidentes[drivers_reg][index]

    df_window = get_drivers_windowed_events(df_alertas, end_date, drivers_reg, date_col_alertas, driver_id, window)

    if df_window.empty:
        continue
    else:
        df_count_new = df_window[cols_to_add_in_acidentes].sum().astype(int)
        for col in cols_to_add_in_acidentes:
            df_count[col][index] = df_count_new[col]


4970 of 4970


In [12]:
df_count.iloc[50]

SIN                               892021
classe_evento_velocidade               0
classe_evento_curva                   11
classe_evento_freio                    0
classe_evento_aceleracao               0
classe_evento_outro                    0
classe_evento_freio_motor              0
classe_evento_direcao_perigosa         0
classe_evento_cambio                   0
classe_evento_autenticacao             0
velocidade_acima_de_80_kmph            0
velocidade_de_50_ate_80_kmph           8
velocidade_de_30_ate_50_kmph           3
velocidade_ate_30_kmph                 0
velocidade_N/I                         0
velocidade_parado                      0
Name: 50, dtype: object